# Параллельное и многопоточное программирование в Python

### Процесс

**Процесс** - программа, которая запущена в оперативной памяти компьютера.

**Процесс** — экземпляр программы во время выполнения, независимый объект, которому выделены системные ресурсы (например, процессорное время и память). Каждый процесс выполняется в отдельном адресном пространстве: один процесс не может получить доступ к переменным и структурам данных другого.

Характеристики:
- PID
- Оперативная память
- Стек
- Список открытых файлов
- Ввод и вывод

In [3]:
import os
import time

#Получить pid процесса
pid = os.getpid()

for i in range(5):
    print(pid, time.time())
    time.sleep(1)

29632 1542122183.9130654
29632 1542122184.9146926
29632 1542122185.9162965
29632 1542122186.9178855
29632 1542122187.9192789


In [8]:
# Создание процесса
from multiprocessing import Process
from functions import say_hello

#Как выглядит функция в самом файле:
# def say_hello(name):
#     print(os.getpid())
#     print("hello", name)
 
p = Process(target = say_hello, args = ('Vladimir',)) 

p.start()
# Завершает открытый процесс, необходимо контролировать ресуры машины 
p.join()


31148
hello Vladimir


In [9]:
# Создание процесса - альтернативный способ
from functions import say_hello_class

#Как выглядит  в самом файле:

# class say_hello_class(Process):
#     def __init__(self, name):

# Явно вызываем родительский конструктор.
# Его главная задача это возможность использования в классе потомке, методов класса-родителя.

#         super().__init__()
#         self.name = name
        
#     def run(self):
#         print(os.getpid())
#         print("hello", self.name)
        
p =  say_hello_class('Vladimir')
p.start()
p.join()

32249
hello Vladimir


### Поток

- все потоки выполняются в рамках одного процесса
- потоки разделяют память и ресурсы процесса, в котором они работают

Поток != Процесс:
- у процесса своё адресное пространство
- у потоков – разделяемое
- у процесса есть хотя бы один поток (Main)
- 1 процесс может содержать несколько
потоков

In [10]:
# Создание потока
from threading import Thread
 
t = Thread(target = say_hello, args = ('Vladimir',)) 

t.start()
t.join()

29632
hello Vladimir


In [12]:
# Создание потока - альтернативный способ
from functions import say_hello_class_T

#Как выглядит  в самом файле:
# class say_hello_class_T(Thread):
#     def __init__(self, name):
#         super().__init__()
#         self.name = name
        
#     def run(self):
#         print(os.getpid())
#         print("hello", self.name)
        
p =  say_hello_class_T('Vladimir')
p.start()
p.join()

29632
hello Vladimir


<img width = '700px' src="images/lesson_9/mini_magick20171116-124-1elj0oj.png">

- Конкуренция - совместная работа (процессор очень быстро переключается между задачами)
- Параллелизм - параллельная работа (задача решается на нескольких ядрах)

### Многопоточность

Сравним две функции.

In [13]:
%%time
import datetime
from time import sleep

def slow_print(s):
    print('Вызов {} в {}'.format(s, datetime.datetime.now()))
    sleep(1)
    print('Конец {} в {}'.format(s, datetime.datetime.now()))

for i in range(5):
    slow_print(i) 

Вызов 0 в 2018-11-13 18:33:50.329107
Конец 0 в 2018-11-13 18:33:51.330595
Вызов 1 в 2018-11-13 18:33:51.330878
Конец 1 в 2018-11-13 18:33:52.332275
Вызов 2 в 2018-11-13 18:33:52.332562
Конец 2 в 2018-11-13 18:33:53.333704
Вызов 3 в 2018-11-13 18:33:53.334080
Конец 3 в 2018-11-13 18:33:54.335283
Вызов 4 в 2018-11-13 18:33:54.335571
Конец 4 в 2018-11-13 18:33:55.336669
CPU times: user 14.4 ms, sys: 3.72 ms, total: 18.1 ms
Wall time: 5.01 s


In [14]:
from threading import Thread

for i in range(5):
    t = Thread(target=slow_print,args=(i,))
    t.start()
    
#ждет, пока тред закончит выполняться    
#здесь завершать не нужно, так как тред завершается сам
#     t.join()

Вызов 0 в 2018-11-13 18:33:55.344168
Вызов 1 в 2018-11-13 18:33:55.344527
Вызов 2 в 2018-11-13 18:33:55.344859
Вызов 3 в 2018-11-13 18:33:55.345374
Вызов 4 в 2018-11-13 18:33:55.345863
Конец 0 в 2018-11-13 18:33:56.345453
Конец 2 в 2018-11-13 18:33:56.347047
Конец 4 в 2018-11-13 18:33:56.348126
Конец 1 в 2018-11-13 18:33:56.349282
Конец 3 в 2018-11-13 18:33:56.349581


<img width = '700px' src="images/lesson_9/multi.png">

Вычислительные задачи можно разделить на две категории:

- I/O bound - требующие ввода/вывода
- CPU bound - нагружающие процессор

### GIL

Глобальная блокировка интерпретатора ( global interpreter lock , **GIL** ) Python. Если два или более потока попытаются манипулировать одним и тем же объектом в одно и то же время, то неизбежно возникнут проблемы. Глобальная блокировка интерпретатора исправляет это. В любой момент времени действия может выполнять только один поток. Python автоматически переключается между потоками, когда в этом возникает необходимость.

Сам интерпретатор CPython нe является потокобезопасным, поэтому в нем есть глобальная блокировка интерпретатора (Global Interpreter Lock - GIL), которая разрешает в каждый момент времени выполнятьбайт-код только одному потоку. Именно поэтому один процесс Python обычно не может задействовать несколько процессорных ядер одновременно.

GIL в Python реализован как обычный lock.

<img width = '700px' src="images/lesson_9/thread.png">

### Упражнение

Написать класс, который многопотчно скачивает документы из интернета и выполнить с помощью него многопоточную загрузку.

In [15]:
import urllib.request
# Список документов, которые необходимо скачать.

urls = ["https://cs8.pikabu.ru/post_img/2017/12/22/11/1513967276125723598.jpg",
        "https://cs7.pikabu.ru/post_img/2017/12/22/10/1513964378174980200.jpg",
        "https://cs8.pikabu.ru/post_img/2017/12/22/10/151396556419317615.jpg",
        "https://cs8.pikabu.ru/post_img/2017/12/22/11/1513967585163753346.jpg",
        "https://cs8.pikabu.ru/post_img/2017/12/22/11/1513965971173110667.jpg",
        "https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966565120257829.jpg",
        "https://cs8.pikabu.ru/post_img/2017/12/22/11/1513966490163375420.jpg",
        "https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966594199245649.jpg",
        "https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966683175659107.jpg"]

1. Создать класс с названием MultiThreadLoader, он должен быть потомком класса Thread.

2. Констурктор класса в качестве атрибутов должен принимать название загружаемого документа(url), а также имя загружамеого документа(name).

Не забыть вызвать конструктор родительского класса, а также назначить имя и название загружаемого документа.

3. Переназначить функцию run родительского класса, в которой должна происходить загрузка.

In [16]:
# Рекомендация по созданию функции загрузки:

# открыть url:
handle = urllib.request.urlopen( )

# из url получить название файла:
fname = os.path.basename( )
        
# осуществить чтение файла из интернета чанками:        
with open(fname, "wb") as f_handler:
    while True:
        chunk = handle.read(1024)
        if not chunk:
            break
        f_handler.write(chunk)
        
#добавить сообщение о том, что конкретный файл загружен        

TypeError: urlopen() missing 1 required positional argument: 'url'

4. Создать цикл, в котором запускаются потоки с помщью метода start, тем самым осуществляется их параллельный запуск. Каждому потоку назначается загрузка отдельного файла картинки.

In [17]:
class MultiThreadLoader(Thread):
    
    def __init__(self, url, name):
        super().__init__()
        self.name = name
        self.url = url
        
    def run(self):
        handle = urllib.request.urlopen(self.url)
        fname = os.path.basename(self.url)
        
        with open(fname, "wb") as f_handler:
            while True:
                chunk = handle.read(1024)
                if not chunk:
                    break
                f_handler.write(chunk)
                
        msg = "%s has finished downloading %s" % (self.name,
                                                   self.url)
        print(msg)          

In [21]:
for item, url in enumerate(urls):
    name = "Thread %s" % (item+1)
    thread = MultiThreadLoader(url, name)
    thread.start()    

Thread 6 has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966565120257829.jpg
Thread 2 has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/10/1513964378174980200.jpg
Thread 8 has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966594199245649.jpg
Thread 9 has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966683175659107.jpg
Thread 5 has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513965971173110667.jpg
Thread 7 has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513966490163375420.jpg
Thread 1 has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513967276125723598.jpg
Thread 3 has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/10/151396556419317615.jpg
Thread 4 has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513967585163753346.jpg


### Синхронизация доступа

При использовании более чем одного треда может возникнуть проблема, когда треды обращаются к одному ресурсу. 

In [42]:
import threading 

def first_operation():
    global x
    while x < 3000000:
        x += 1
    print (x)    
    
def second_operation():
    global x
    while x < 6000000:
        x += 1
    print (x)        

global x 
x = 0 

thread_1 =   threading.Thread(target = first_operation)  
thread_1.start()

thread_2 =   threading.Thread(target = second_operation)  
thread_2.start()

3000000
6000000


Как такое могло произойти? - Оба треда имели доступ к глобальной переменной x.

### Таймер

Начинает выполнение функции через определенный промежуток времени.

In [43]:
def first_operation():
    global x
    while x < 3000000:
        x += 1
    print (x)    
    
def second_operation():
    global x
    while x < 6000000:
        x += 1
    print (x)        

global x 
x = 0 

thread_1 =   threading.Thread(target = first_operation)  
thread_1.start()

thread_2 =   threading.Timer(1,function = second_operation)  
thread_2.start()

3000000
6000000


### Замки

In [44]:
def first_operation():
    global x,lock
    lock.acquire()
    try:
        while x < 3000000:
            x += 1
        print (x)   
    finally:
        lock.release()
    
def second_operation():
    global x,lock
    lock.acquire()
    try:
        while x < 6000000:
            x += 1
        print (x)   
    finally:
        lock.release()      

global x, lock 
x = 0 

lock = threading.Lock()

thread_1 =   threading.Thread(target = first_operation)  
thread_1.start()

thread_2 =   threading.Thread(target = second_operation)  
thread_2.start()

3000000
6000000


### Дедлоки

In [16]:
a = threading.Lock()
b = threading.Lock()

#неправильный порядок освобождения лока может привести к дедлоку
def foo():
    try:
        a.acquire()
        b.acquire()
    finally:
        a.release()
        b.release()

### Семафоры

Семафоры - более сложный и совершенный механизм блокировок. Внутри семафора - счетчик, в отличии от объекта блокировки, в которой просто флажок. Семафор блокирует поток только когда более заданного числа потоков пытаются захватить семафор.

In [49]:
def first_operation():
    global x,semaphore
    semaphore.acquire()
    try:
        while x < 3000000:
            x += 1
        print (x)   
    finally:
        semaphore.release()
    
def second_operation():
    global x,semaphore
    semaphore.acquire()
    try:
        while x < 6000000:
            x += 1
        print (x)   
    finally:
        semaphore.release()      

global x, semaphore 
x = 0 

# semaphore = threading.BoundedSemaphore(10) 
semaphore = threading.BoundedSemaphore(1) 

thread_1 =   threading.Thread(target = first_operation)  
thread_1.start()

thread_2 =   threading.Thread(target = second_operation)  
thread_2.start()

3000000
6000000


Прочие методы - RLock, Events, Conditions, Queue. Прочитать - http://www.quizful.net/post/thread-synchronization-in-python.

### multiprocessing

In [103]:
from functions import count
# def count(n):
#     while n > 0:
#         n -= 1

In [51]:
# cpu bound задача
t0 = time.time()
count(100000000)
count(100000000)
print(time.time() - t0)

# parallel run
t0 = time.time()
th1 = Thread(target=count, args=(100000000,))
th2 = Thread(target=count, args=(100000000,))

th1.start(); th2.start()
th1.join(); th2.join()
print(time.time() - t0)

6.909359693527222
7.069227695465088


In [52]:
# process parallel run
#разобьем задачу на процессы
t0 = time.time()
th1 = Process(target=count, args=(100000000,))
th2 = Process(target=count, args=(100000000,))

th1.start(); th2.start()
th1.join(); th2.join()
print(time.time() - t0)

3.4332332611083984


### Замки

Аналогично тредам. Процесс ждет открытия замка.

In [53]:
from functions import printer
from multiprocessing import Lock

# def printer(item, lock):
#     proc = os.getpid()
#     print('Вызов {} в {} процесс {}'.format(item, datetime.datetime.now(),proc))
#     lock.acquire()
#     try:
#         print(item)
#         sleep(1)
#     finally:
#         lock.release()
#     print('Конец {} в {} процесс {}'.format(item, datetime.datetime.now(), proc))

In [54]:
lock = Lock()
items = ['tango', 'foxtrot', 10]
for item in items:
    p = Process(target=printer, args=(item, lock))
    p.start()

Вызов tango в 2018-11-13 19:25:32.211835 процесс 13175
Вызов foxtrot в 2018-11-13 19:25:32.215844 процесс 13176
tango
Вызов 10 в 2018-11-13 19:25:32.219711 процесс 13183
Конец tango в 2018-11-13 19:25:33.219723 процесс 13175
foxtrot
Конец foxtrot в 2018-11-13 19:25:34.224820 процесс 13176
10
Конец 10 в 2018-11-13 19:25:35.229765 процесс 13183


### Класс Pool

С помощью данного класса можем назначать процессы на отдельные ядра.

In [23]:
from multiprocessing import Pool
from functions import slow_print

numbers = [5, 10, 20]
pool = Pool(processes=3)

pool.map(slow_print, numbers)

[None, None, None]

### Модуль concurrent.futures

Абстракция над модулями мультипроцессинга и мультипоточности: можно работать с обоими модулями, но при этом не обладает большой гибкостью в управлении.

In [55]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed
import os

In [56]:
def run(url):
    handle = urllib.request.urlopen(url)    
    fname = os.path.basename(url)
    with open(fname, "wb") as f_handler:
        while True:
            chunk = handle.read(1024)
            if not chunk:
                break
            f_handler.write(chunk)
    msg = "%s has finished downloading %s" % (fname,url)
    print(msg)

In [57]:
# from functions import run

   
def pool_mapper(urls):
    with ThreadPoolExecutor(max_workers = 9) as executor:
            return executor.map(run, urls)  
        
pool_mapper(urls)        

1513965971173110667.jpg has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513965971173110667.jpg
1513966594199245649.jpg has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966594199245649.jpg
1513966490163375420.jpg has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513966490163375420.jpg
1513966565120257829.jpg has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966565120257829.jpg
1513967276125723598.jpg has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/11/1513967276125723598.jpg
1513966683175659107.jpg has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/11/1513966683175659107.jpg
151396556419317615.jpg has finished downloading https://cs8.pikabu.ru/post_img/2017/12/22/10/151396556419317615.jpg
1513964378174980200.jpg has finished downloading https://cs7.pikabu.ru/post_img/2017/12/22/10/1513964378174980200.jpg
1513967585163753346.jpg has finished downloading https://c

<generator object Executor.map.<locals>.result_iterator at 0x7f2f4074deb8>

### Упражнение

Оценить качество алгоритмов с помощью коэффициента детерминации путем распараллеливания процессов. 

In [58]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from multiprocessing import Pool

1. Загрузить датасет из файла train.csv. Изучить датасет.

In [59]:
import pandas as pd
data_ = pd.read_csv('files/lesson_9/train.csv')
data_.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


2. Написать функцию calculate_quality, импортируемую из внешнего файла. В функции должны присутсовать следующие этапы:
    - загрузка датасета из файла;
    - разбивка датасета на обучающую и тестовую выборку с помощью фукции train_test_split, обратить внимание, какие столбцы должны использоваться для обучения, а какие для теста, размер тестовой выборки - 33%, random_state = 42; 
    - в качестве атрибута функция должна получать классификатор, который мы будем передавать извне
    - функция должна print`ом выводить коэффициент детерминации, рассчитанный на тестовой выборке.

3. Оценить качество моделей, полученных с помощью алгоритмов RandomForestRegressor и GradientBoostingRegressor, рассчитав качество на 2 ядрах параллельно.

In [86]:
from sklearn.metrics import  r2_score
from sklearn.model_selection import  train_test_split
import numpy as np
from sklearn import ensemble
data_ = pd.read_csv('files/lesson_9/train.csv')
y = data_.target
X = data_.drop(['target', 'ID'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf1 = ensemble.GradientBoostingRegressor()
clf1.fit(X_train, y_train)
print("R^2:", r2_score(y_test, clf1.predict(X_test)))

In [87]:
clf1 = ensemble.GradientBoostingRegressor()

clf1.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [88]:
mse = mean_squared_error(y_test, clf1.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 52111743850635.1641


In [89]:
print("R^2:", r2_score(y_test, clf1.predict(X_test)))

R^2: 0.183508823373698


In [90]:
clf2 = ensemble.RandomForestRegressor()
clf2.fit(X_train, y_train)

/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [91]:
mse = mean_squared_error(y_test, clf2.predict(X_test))
print("MSE: %.4f" % mse)

MSE: 53489676503563.5000


In [92]:
print("R^2:", r2_score(y_test, clf2.predict(X_test)))

R^2: 0.16191925891149195


In [ ]:
%%time
def pool_mapper_two(urls):
    with ThreadPoolExecutor(max_workers = 3) as executor:
            return executor.map(second, urls)  
        
pool_mapper_two(urls)

/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/evgenii/.local/lib/python3.6/site-packages/skl

In [2]:
from func import calculate_quality

In [3]:
from sklearn import ensemble
from multiprocessing import Pool
from functions import slow_print
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import as_completed
import os
classifiers_list = [ensemble.RandomForestRegressor(), ensemble.GradientBoostingRegressor()]
#Обычный Pool
pool = Pool(processes = 2)
pool.map(calculate_quality, classifiers_list)

def pool_mapper(classifiers_list):
    with ProcessPoolExecutor(max_workers=2) as executor:
            return executor.map(calculate_quality, classifiers_list, timeout=60)  
        
pool_mapper(classifiers_list)

/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/evgenii/ADMIN_INFO/technosphere/Python/9/func.py", line 11, in calculate_quality
    clf.fit(X_train, y_train)
  File "/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/gradient_boosting.py", line 1465, in fit
    begin_at_stage, monitor, X_idx

KeyboardInterrupt: 

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/evgenii/ADMIN_INFO/technosphere/Python/9/func.py", line 11, in calculate_quality
    clf.fit(X_train, y_train)
  File "/home/evgenii/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py", line 335, in fit
    for i, t in enumerate(trees))
  File "/home/evgenii/.local/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 983, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/evgenii/.local/lib/python3.6/site-packages/sklearn/externals/joblib/paralle